# Text Summarization

Урок 12. Модель Transformer-2
Задание
Реализовать суммаризацию текста
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
Проверить насколько хорошо она суммаризирует

In [1]:
import numpy as np
import pandas as pd

In [2]:
from collections import Counter

In [3]:
from itertools import combinations
import networkx as nx
import pymorphy2
import numpy as np
from tqdm.notebook import tqdm

In [4]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
df_rec = pd.read_csv('../data/all_recepies_inter.csv', sep='\t')

In [6]:
df_rec['name'][:4]

0    рассольник классический с перловкой и солеными...
1                      Суп пюре из белокочаной капусты
2                       Постные щи из квашеной капусты
3                    Тюря- простой суп быстро и вкусно
Name: name, dtype: object

In [7]:
df_rec.shape

(27884, 10)

In [8]:
df_rec.rename(columns={"Инструкции": "text"}, inplace=True)

In [10]:
data_5000 = df_rec.loc[:5000, 'text']
data_all = df_rec['text']
summary_5000 = df_rec.loc[:5000, 'name']
summary_all = df_rec['name']

In [11]:
# Делаем предобработку
import spacy
import razdel

In [12]:
# Список частей речи, которые мы не хотим считать значимыми.
# Подбирался на глаз.
BAD_POS = ("PREP", "NPRO", "CONJ", "PRCL", "NUMR", "PRED", "INTJ", "PUNCT", "CCONJ", "ADP", "DET", "ADV")

#not very good results, so we'll try concentrating on nouns which seems better for recipes
GOOD_POS = "NOUN"

# Загрузка модели для частеречной разметки.
spacy_model = spacy.load("ru_core_news_md")


# Метод для разбиения текста на предложения.
def sentenize(text):
    return [s.text for s in razdel.sentenize(text)]


# Метод для токенизации предложения.
def tokenize_sentence(sentence):
    sentence = sentence.strip().replace("\xa0", "")
    tokens = [token.lemma_ for token in spacy_model(sentence) if token.pos_ == GOOD_POS]
    tokens = [token for token in tokens if len(token) > 2]
    return tokens



# Метод для токенизации всего текста.
def tokenize_text(text):
    all_tokens = []
    for sentence in sentenize(text):
        all_tokens.extend(tokenize_sentence(sentence))
    return all_tokens



In [13]:
# Пример работы обоих методов.
text = data_5000[0]
sentences = sentenize(text)
print(tokenize_sentence(sentences[0]))
print(tokenize_text(text))

['ингредиент', 'приготовление', 'рассольник', 'крупа']
['ингредиент', 'приготовление', 'рассольник', 'крупа', 'бульон', 'говядина', 'курица', 'суп', 'вода', 'рассольник', 'паста', 'разнообразие', 'желание', 'специя', 'соль', 'перец', 'горошек', 'перец', 'крупа', 'вода', 'бульон', 'перловка', 'среднее', 'огонь', 'рассольник', 'огурец', 'огурец', 'рассольник', 'рассол', 'огурец', 'огурец', 'рассол', 'тёрка', 'картофель', 'кубик', 'очередь', 'картофель', 'вода', 'морковь', 'лук', 'чеснок', 'морковь', 'тёрка', 'лук', 'сельдерей', 'чеснок', 'мелко', 'масло', 'овощ', 'паста', 'паста', 'желание', 'минута', 'крупа', 'картофель', 'среднее', 'огонь', 'минута', 'суп', 'овощ', 'вкус', 'минута', 'огурец', 'суп', 'перец', 'горошек', 'рассольник', 'огурец', 'минута', 'нагрев', 'зелень', 'супу', 'плита', 'минута', 'подача', 'рассольник', 'тарелка', 'сметана', 'зелень']


In [14]:
sentences

['Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой.',
 'Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде.',
 'Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию.',
 'Из специй соль, чёрный перец горошком, душистый перец.',
 'Перловую крупу промыть до чистой воды.',
 'В горячий бульон добавить промытую перловку и варить на среднем огне.',
 'Для рассольника лучше брать кислые, очень солёные огурцы.',
 'Если же огурцы обычные, то рекомендуется в рассольник добавлять из сам рассол от огурцов.',
 'Солёные огурцы достать из рассола и натереть на крупной тёрке.',
 'Картофель помыть, обсушить, очистить.',
 'Нарезать кубиками.',
 'Пока очередь картофеля не подошла, положить его в воду.',
 'Морковь, лук, чеснок очистить.',
 'Морковь натереть на крупной тёрке, лук, сельдерей, чеснок мелко порезать.',
 'Обжарить в масле овощи, добавив томатной пасты.',
 'То

In [15]:
from collections import Counter

class LuhnSummarizer:
    """
    Метод Луна.
    Основано на https://github.com/miso-belica/sumy/blob/main/sumy/summarizers/luhn.py
    Оригинальная статья: https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf
    """
    def __init__(
        self,
        significant_percentage = 0.4, # 40% самых частотных токенов мы считаем значимыми.
        min_token_freq = 4, # Кроме того, слова должны встречаться минимум 2 раза.
        max_gap_size = 5, # Максимальное количество подряд идущих незначимых токенов в промежутках.
        verbose = False # Отладочный вывод для наглядности.
    ):
        self.significant_percentage = significant_percentage
        self.min_token_freq = min_token_freq
        self.max_gap_size = max_gap_size
        self.chunk_ending_mask = [0] * self.max_gap_size
        self.verbose = verbose

    def __call__(self, text, target_sentences_count):
        # Считаем значимые токены.
        all_significant_tokens = self._get_significant_tokens(text)
        if self.verbose:
            print("Значимые токены: ", all_significant_tokens)

        # Считаем значимости предложений.
        ratings = []
        for sentence_index, sentence in enumerate(sentenize(text)):
            # Значимость предложений - максимум из значимостей промежутков.

            all_rates = self._get_chunk_ratings(sentence, all_significant_tokens)
    # checking chunk validity
            if all_rates:
                sentence_rating = max(all_rates)
            if self.verbose:
                print("\tПРЕДЛОЖЕНИЕ. Значимость: {}, текст: {}".format(sentence_rating, sentence))
            ratings.append((sentence_rating, sentence_index))

        # Сортируем предложения по значимости.
        ratings.sort(reverse=True)

        # Оставляем топовые и собираем реферат.
        ratings = ratings[:target_sentences_count]
        indices = [index for _, index in ratings]
        indices.sort()
        return " ".join([sentences[index] for index in indices])

    def _get_significant_tokens(self, text):
        """ Метод для подсчёта того, какие токены являются значимыми. """
        tokens_counter = Counter(tokenize_text(text))
        significant_tokens_max_count = int(len(tokens_counter) * self.significant_percentage)
        significant_tokens = tokens_counter.most_common(significant_tokens_max_count)
        significant_tokens = {token for token, cnt in significant_tokens if cnt >= self.min_token_freq}
        return significant_tokens

    def _get_chunk_ratings(self, sentence, significant_tokens):
        """ Разбиваем предложение на промежтуки и считаем их значимости. """

        tokens = tokenize_sentence(sentence)

        chunks, masks = [], []
        in_chunk = False
        for token in tokens:
            is_significant_token = token in significant_tokens
            
            if is_significant_token and not in_chunk:
                in_chunk = True
                masks.append([int(is_significant_token)])
                chunks.append([token])
            elif in_chunk:
                last_mask = masks[-1]
                last_mask.append(int(is_significant_token))
                last_chunk = chunks[-1]
                last_chunk.append(token)
            if not chunks:
                continue

            # Проверяем на наличие 4 подряд идущих незначимых токенов.
            # Если встретили - завершаем промежуток.
            last_chunk_ending_mask = masks[-1][-self.max_gap_size:]
            if last_chunk_ending_mask == self.chunk_ending_mask:
                in_chunk = False
        
        ratings = []
        for chunk, mask in zip(chunks, masks):
            rating = self._get_chunk_rating(mask, chunk)
            ratings.append(rating)
        return ratings

    def _get_chunk_rating(self, original_mask, chunk): 
        """ Подсчёт значимости одного промежутка """

        # Убираем незначимые токены в конце промежутка
        original_mask = "".join(map(str, original_mask))
        mask = original_mask.rstrip("0")

        end_index = original_mask.rfind("1") + 1
        chunk = chunk[:end_index]
        assert len(mask) == len(chunk)
        chunk = " ".join(chunk)

        # Считаем значимость
        words_count = len(mask)
        assert words_count > 0
        significant_words_count = mask.count("1")
        assert significant_words_count > 0

        rating = significant_words_count * significant_words_count / words_count
        if self.verbose:
            print("ПРОМЕЖУТОК. Значимость: {}, маска: {}, текст: {}".format(rating, mask, chunk))
        return rating

In [16]:
luhn = LuhnSummarizer(verbose=False)
summary = luhn(text, 1)
print()
print("Текст: {}".format(data_5000[0]))
print("Итоговый реферат: {}".format(summary))
#print("Правильный реферат: {}".format(test_records[0]["summary"]))


Текст: Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой. Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде. Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию. Из специй соль, чёрный перец горошком, душистый перец.
Перловую крупу промыть до чистой воды.
В горячий бульон добавить промытую перловку и варить на среднем огне.
Для рассольника лучше брать кислые, очень солёные огурцы. Если же огурцы обычные, то рекомендуется в рассольник добавлять из сам рассол от огурцов. Солёные огурцы достать из рассола и натереть на крупной тёрке.
Картофель помыть, обсушить, очистить. Нарезать кубиками. Пока очередь картофеля не подошла, положить его в воду.
Морковь, лук, чеснок очистить. Морковь натереть на крупной тёрке, лук, сельдерей, чеснок мелко порезать.
Обжарить в масле овощи, добавив томатной пасты. Томатная паста по желанию.
Минут через двадцать добавить

## TextRank

https://habr.com/ru/post/455762/

**don't think it'll work, for the name of the recipe is usually of most relevant words, not sentences**

### Самописный TextRank

In [17]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [18]:
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2)) / (len(words1) + len(words2))
    #return len(words1.intersection(words2)) / (np.log10(len(words1)) + np.log10(len(words2)))

def gen_text_rank_summary(text, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    
    pr = nx.pagerank(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = 1
    
    result = result[:n_summary_sentences]
    
    # Восстанавливаем оригинальный их порядок
    #result.sort(key=lambda x: x[0])
    
    #num_sents = len(result)
    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=unique_words_similarity, summary_part=0.1, lower=True, nrows=1000, morph=None):
    references = []
    predictions = []
  #  length_sents = [] 

    for i in tqdm(records.index):
        if i >= nrows:
            break

        summary = records["name"][i]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = records["text"][i]
        predicted_summary = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
   #     length_sents.append(num_sents)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)
        print(f' preds {predicted_summary}')

 #   print(f'Mean number of sents in summary: {np.mean(length_sents)}')
    calc_scores(references, predictions)
    
    return predictions


In [19]:
morph = pymorphy2.MorphAnalyzer()
preds = calc_text_rank_score(df_rec[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

 preds морковь, лук, чеснок очистить.
 preds необходимые ингредиенты
нарезаем лук, морковь и чеснок, овощи можно особо не мельчить потому как все будем превращать в пюре
пассеруем морковь и лук на оливковом масле на сильном огне буквально пару минут, добавляем чеснок, перемешиваем и снимаем с огня
пока овощи обжариваются, крупно нарезаем картошку
выкладываем обжаренные овощи в кипящую воду
сюда же кладем картошку
нарезаем крупно капусту
кладем капусту в кастрюлю к остальным овощам
солим
перчим и варим 20-25 минут
в конце добавляем лавровый лист, выключаем огонь и даем настояться 5 минут
превращаем овощи в пюре любым удобным и доступным способом
подаем с сухариками и свежей зеленью.
 preds на сковороде прогрейте муку и мак, разведите парой ложек воды, добавьте чеснок, размешайте.
 preds вот и все!
 preds добавить в кастрюлю к фасоли картофель и морковь.
 preds морковь моем, чистим и натираем на терке.
 preds затем добавляем к луку томатное пюре и молотый перец чили, все перемешиваем и п

*it truly didn't*

## Lexrank
* Original paper: https://arxiv.org/pdf/1109.2128.pdf
* lexrank library: https://github.com/crabcamp/lexrank

In [20]:
import lexrank
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS

In [21]:
def calc_method_score(records, predict_func, nrows=1000):
    references = []
    predictions = []

    for i in tqdm(records.index):
        if nrows is not None and i >= nrows:
            break
        summary = records["name"][i]
        text = records["text"][i]
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)

    calc_scores(references, predictions)
    return predictions

In [22]:
def predict_lex_rank(text, summary, lxr, summary_size=1, threshold=None):
    sentences = [s.text for s in razdel.sentenize(text)]
    #print(f'sentenized {sentences}')
    prediction = lxr.get_summary(sentences, summary_size=summary_size, threshold=threshold)
    prediction = " ".join(prediction)
   # print(f'pred {prediction}')
    return prediction

In [23]:
sentences = [[s.text for s in razdel.sentenize(data_5000[i])] for i in range(len(data_5000))]
lxr = LexRank(sentences, stopwords=STOPWORDS['ru'])
preds_lxr = calc_method_score(df_rec[:10], lambda x, y: predict_lex_rank(x, y, lxr))

  0%|          | 0/10 [00:00<?, ?it/s]

Count: 10
Ref: Томатный суп пюре классический
Hyp: Пока томаты в воде, очистить луковицу, промыть и нарезать мелким кубиком.
BLEU:  0.10284477799563467
ROUGE:  {'rouge-1': {'r': 0.12357142857142855, 'p': 0.0493939393939394, 'f': 0.0677991433647021}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.12357142857142855, 'p': 0.0493939393939394, 'f': 0.0677991433647021}}


In [24]:
print(preds_lxr)

['При подаче в рассольник в каждую тарелку положить свежую сметану и зелень.', 'Приятного аппетита!', 'Они должны жариться на небольшом огне, до прозрачности.Отдельно потушим квашенную капусту в глубокой сковороде, залив капусту водой.', 'Затем нарезаем наш хлебушек некрупными кубиками и выкладываем на тарелку.', 'В кастрюлю выложить фасоль, влить 2 л воды и варить до готовности фасоли около 2 часов.', 'Пока варится борщ, приготовим тесто на пампушки.', 'Холодный или предварительно подогретый говяжий бульон наливаем в кастрюлю, где тушились овощи, все пемешиваем и доводим до кипения при нечастом помешивании.', 'Затем добавляем в суп мяту и солим по вкусу.Накрываем кастрюлю крышкой и готовим на минимальном огне, пока чечевица и булгур не станут мягкими.', 'Когда горох будет практически готов, добавьте измельченный картофель в суп.', 'Пока томаты в воде, очистить луковицу, промыть и нарезать мелким кубиком.']


## LSA
* One of the original papers: https://www.cs.bham.ac.uk/~pxt/IDA/text_summary.pdf
* sumy library: https://github.com/miso-belica/sumy

In [25]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import nltk; nltk.download('punkt');


def predict_lsa(text, summary, lsa_summarizer, tokenizer, summary_size=1):
    parser = PlaintextParser.from_string(text, tokenizer)
    predicted_summary = lsa_summarizer(parser.document, summary_size)
    predicted_summary = " ".join([str(s) for s in predicted_summary])
    return predicted_summary

lsa_summarizer = LsaSummarizer()
tokenizer = Tokenizer("russian")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Armik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
lsa_preds = calc_method_score(df_rec[:10], lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer))

  0%|          | 0/10 [00:00<?, ?it/s]

Count: 10
Ref: Томатный суп пюре классический
Hyp: Если сначала дать массе остыть, а потом пюрировать ее, то после измельчения суп необходимо будет еще раз прогреть до кипения.
BLEU:  0.05387095614825197
ROUGE:  {'rouge-1': {'r': 0.11595238095238095, 'p': 0.028241327300150824, 'f': 0.043486084378749026}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.09595238095238094, 'p': 0.024074660633484162, 'f': 0.03658953265461111}}


In [27]:
lsa_preds

['Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию.',
 'Необходимые ингредиенты Нарезаем лук, морковь и чеснок, овощи можно особо не мельчить потому как все будем превращать в пюре Пассеруем морковь и лук на оливковом масле на сильном огне буквально пару минут, добавляем чеснок, перемешиваем и снимаем с огня Пока овощи обжариваются, крупно нарезаем картошку Выкладываем обжаренные овощи в кипящую воду Сюда же кладем картошку Нарезаем крупно капусту Кладем капусту в кастрюлю к остальным овощам Солим Перчим и варим 20-25 минут В конце добавляем лавровый лист, выключаем огонь и даем настояться 5 минут Превращаем овощи в пюре любым удобным и доступным способом Подаем с сухариками и свежей зеленью.',
 'Выложите пассированные овощи к капусте, размешайте и потушите все вместе полчаса.Переложите массу в кастрюлю на 3 литра, влейте 1,5-2 литра горячей воды (я добавляла картофельный отвар с 4 толчеными клубнями картошки) и подождите, пока жидко

### Summa

In [28]:
from summa.summarizer import summarize

In [30]:
def calc_summa_score(records, words=5, lower=True, nrows=1000):
    references = []
    predictions = []

    for i in tqdm(records.index):
        if nrows is not None and i >= nrows:
            break
        summary = records["name"][i]
        text = records["text"][i]

        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = text if not lower else text.lower()
        predicted_summary = summarize(text, words=words, language='russian').replace("\n", " ")
        predictions.append(predicted_summary)

    #calc_scores(references, predictions)
    return predictions


In [31]:
preds_sum = calc_summa_score(df_rec[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [32]:
preds_sum

['',
 '',
 '',
 'затем нарезаем наш хлебушек некрупными кубиками и выкладываем на тарелку.',
 '',
 '',
 '',
 '',
 '',
 '']

## Oracle summary

Для сведения задачи к extractive summarization мы должны выбрать те предложения из оригинального текста, которые наиболее похожи на наше целевое summary по нашим метрикам.

In [33]:
import copy

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    
    score = -1.0
    summaries = []
    for _ in range(n_sentences):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [34]:
from tqdm.notebook import tqdm
import razdel

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for i in tqdm(records.index):
        if nrows is not None and i >= nrows:
            break
        summary = records["name"][i]
        text = records["text"][i]

        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)
    return predictions


preds_oracle = calc_oracle_score(df_rec[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

Count: 10
Ref: томатный суп пюре классический
Hyp: подготовить указанные продукты.
BLEU:  0.0868523309109819
ROUGE:  {'rouge-1': {'r': 0.2842857142857143, 'p': 0.08551587301587302, 'f': 0.12351072674043576}, 'rouge-2': {'r': 0.18333333333333332, 'p': 0.04575163398692811, 'f': 0.07031745938462587}, 'rouge-l': {'r': 0.2509523809523809, 'p': 0.07996031746031747, 'f': 0.11398691721662624}}


In [35]:
preds_oracle

['подготовить указанные ингредиенты для приготовления рассольника с перловой крупой.',
 'необходимые ингредиенты\r\nнарезаем лук, морковь и чеснок, овощи можно особо не мельчить потому как все будем превращать в пюре\r\nпассеруем морковь и лук на оливковом масле на сильном огне буквально пару минут, добавляем чеснок, перемешиваем и снимаем с огня\r\nпока овощи обжариваются, крупно нарезаем картошку\r\nвыкладываем обжаренные овощи в кипящую воду\r\nсюда же кладем картошку\r\nнарезаем крупно капусту\r\nкладем капусту в кастрюлю к остальным овощам\r\nсолим\r\nперчим и варим 20-25 минут\r\nв конце добавляем лавровый лист, выключаем огонь и даем настояться 5 минут\r\nпревращаем овощи в пюре любым удобным и доступным способом\r\nподаем с сухариками и свежей зеленью.',
 'честно признаюсь, у меня не было репы на момент приготовления, но я прекрасно обошлась и без нее.возьмем овощи: морковку, репу и корень петрушки, обмоем, и очистим.',
 'начинаем мы приготовление тюри с того, что с заранее под